In [5]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

In [6]:
card_ref = pd.read_csv('data_with_expert_var.csv')

### Drop OutofTime data

In [7]:
fraud = card_ref[['Fraud']]

In [4]:
card_ref.drop(['Recnum'],axis = 1,inplace = True)
card_ref.drop(['Unnamed: 0'],axis = 1,inplace = True)
card_ref.drop(['Transtype'],axis = 1,inplace = True)
card_ref.drop(['Merchnum'],axis = 1,inplace = True)
card_ref.drop(['Merch state'],axis = 1,inplace = True)
card_ref.drop(['Merch zip'],axis = 1,inplace = True)
card_ref.drop(['Cardnum'],axis = 1,inplace = True)
card_ref.drop(['Merch description'],axis = 1,inplace = True)
card_ref.drop(['Date'],axis = 1,inplace = True)

In [89]:
#z scale data 
card_ref_scale = card_ref.loc[:,card_ref.columns !='Fraud'].apply(zscore)

In [90]:
card_ref = pd.concat([card_ref_scale,fraud],axis='columns')

In [92]:
#last step
card_ref['RANDOM'] = np.random.ranf(len(card_ref))

In [94]:
card_ref.columns

Index(['Amount', 'avg_card_0', 'avg_card_1', 'avg_card_3', 'avg_card_7',
       'avg_card_14', 'avg_card_30', 'max_card_0', 'max_card_1', 'max_card_3',
       ...
       'vcv_am1_nm14', 'vcv_am1_nm30', 'vcv_am1_ac7', 'vcv_am1_ac14',
       'vcv_am1_ac30', 'vcv_am1_am7', 'vcv_am1_am14', 'vcv_am1_am30', 'Fraud',
       'RANDOM'],
      dtype='object', length=374)

In [95]:
card_ref.shape[0]

83970

In [96]:
card_ref.shape[1]

374

In [ ]:
oot_data = card_ref[card_ref_scale.Recnum>84299]
card_ref = card_ref[card_ref_scale.Recnum<=84299]

In [121]:
card_ref.to_csv('data_expert_var_scale.csv')

In [8]:
card_ref = pd.read_csv('scaled_modeling_data.csv')

In [9]:
del card_ref['Date']

In [10]:
card_ref['RANDOM'] = np.random.ranf(len(card_ref))

In [12]:
del card_ref['Unnamed: 0']

In [13]:
card_ref.head()

,Amount,avg_card_0,avg_card_1,avg_card_3,avg_card_7,avg_card_14,avg_card_30,max_card_0,max_card_1,max_card_3,...,vcv_am1_nm14,vcv_am1_nm30,vcv_am1_ac7,vcv_am1_ac14,vcv_am1_ac30,vcv_am1_am7,vcv_am1_am14,vcv_am1_am30,Fraud,RANDOM
0,-0.471256,-0.504523,-0.546231,-0.600749,-0.676036,-0.720684,-0.782517,-0.457968,-0.487011,-0.528318,...,-0.223322,-0.242971,-0.005725,0.000149,0.001945,0.001255,0.001983,0.000843,0,0.227042
1,-0.437856,-0.468810,-0.507753,-0.558363,-0.628430,-0.669873,-0.727255,-0.432435,-0.464847,-0.508452,...,-0.223322,-0.242971,-0.005725,0.000149,0.001945,0.001255,0.001983,0.000843,0,0.239608
2,-0.261158,-0.279881,-0.304192,-0.334129,-0.376581,-0.401066,-0.434899,-0.297359,-0.347592,-0.403356,...,-0.223322,-0.242971,-0.005725,0.000149,0.001945,0.001255,0.001983,0.000843,0,0.906052
3,-0.471256,-0.504400,-0.546098,-0.600603,-0.675872,-0.720509,-0.782326,-0.457802,-0.486867,-0.528189,...,-0.223322,-0.242971,-0.005725,0.000149,0.001945,0.001255,0.001983,0.000843,0,0.800233
4,-0.471256,-0.504523,-0.546231,-0.600749,-0.676036,-0.720684,-0.782517,-0.457968,-0.487011,-0.528318,...,-0.223322,-0.242971,-0.005725,0.000149,0.001945,0.001255,0.001983,0.000843,0,0.296997


### KS

In [14]:
VarTable = pd.DataFrame(index=range(card_ref.shape[1]),columns=range(3))
VarTable.columns =['Variable','KS','FDR']

In [15]:
VarTable.shape

(374, 3)

In [16]:
i = 0
for col in card_ref:
    VarTable.iloc[i,0] = col
    i+=1

In [17]:
VarTable.head()

,Variable,KS,FDR
0,Amount,NaN,NaN
1,avg_card_0,NaN,NaN
2,avg_card_1,NaN,NaN
3,avg_card_3,NaN,NaN
4,avg_card_7,NaN,NaN


In [18]:
from scipy.stats import ks_2samp
goods = card_ref[card_ref.Fraud == 0]
bads = card_ref[card_ref.Fraud == 1]

In [19]:
i = 0
for col in card_ref:
    VarTable.KS[i] = ks_2samp(goods[col],bads[col])[0]
    i+=1

In [20]:
KS = VarTable.sort_values(['KS'],ascending=False)

In [21]:
KS[KS['Variable']=='days_since_cs']

,Variable,KS,FDR
275,days_since_cs,0.29858,NaN


In [22]:
KS.head()

,Variable,KS,FDR
372,Fraud,1,NaN
187,tot_cz_7,0.674544,NaN
241,tot_cs_3,0.66961,NaN
186,tot_cz_3,0.668773,NaN
242,tot_cs_7,0.666733,NaN


### FDR

In [23]:
i = 0
topn = int(round(len(card_ref)*0.03,0))
for col in card_ref:
    top1 = card_ref.sort_values([col],ascending=True).head(topn)
    top2 = card_ref.sort_values([col],ascending=False).head(topn)
    FDR1 = (top1.Fraud.sum())/len(bads)
    FDR2 = (top2.Fraud.sum())/len(bads)
    VarTable.FDR[i] = max(FDR1,FDR2)
    i+=1

In [24]:
VarTable.head()

,Variable,KS,FDR
0,Amount,0.543573,0.313636
1,avg_card_0,0.539028,0.294318
2,avg_card_1,0.537805,0.323864
3,avg_card_3,0.546009,0.331818
4,avg_card_7,0.505248,0.380682


In [25]:
FDR = VarTable.sort_values(['FDR'],ascending=False).reset_index()

In [26]:
KS = VarTable.sort_values(['KS'],ascending = False).reset_index()

In [27]:
FDR_rank=FDR.reset_index()

In [28]:
del FDR_rank['index']

In [29]:
FDR_rank.columns=['FDR_order','Variable','KS','FDR']

In [30]:
FDR_rank.shape

(374, 4)

In [31]:
KS_rank=KS.reset_index()
del KS_rank['index']
KS_rank.columns=['KS_order','Variable','KS','FDR']

In [32]:
merge_data=FDR_rank.merge(KS_rank,on=['Variable','KS','FDR'])

In [33]:
merge_data = merge_data[['Variable','KS','FDR','KS_order','FDR_order']]

In [34]:
merge_data.shape

(374, 5)

In [35]:
merge_data.to_csv('ks_fdr_table.csv')

In [36]:
merge_data.head()

,Variable,KS,FDR,KS_order,FDR_order
0,Fraud,1,1,0,0
1,tot_cz_3,0.668773,0.628409,3,1
2,tot_cz_7,0.674544,0.623864,1,2
3,tot_cs_3,0.66961,0.621591,2,3
4,tot_cm_7,0.665307,0.611364,5,4


In [37]:
merge_data=merge_data.sort_values(['KS_order'],ascending=True)

# Univariate (KS,FDR): choose half variables

In [38]:
merge_data = merge_data.loc[-merge_data['Variable'].isin(['Fraud','RANDOM']),:]

In [39]:
merge_data['KS_FDR_order_avg'] = (merge_data['KS_order'] + merge_data['FDR_order'])/2

In [40]:
merge_data = merge_data.sort_values(by='KS_FDR_order_avg')

In [43]:
merge_data=merge_data.reset_index()

In [45]:
del merge_data['index']

In [47]:
len(merge_data)/2

186.0

In [50]:
merge_data=merge_data.iloc[0:186,]

In [51]:
merge_data.to_csv('feature_selection_univariate.csv')